Die Ansätze für diese Algorithmen wurden teilweise dem Beitrag von Markus Schmitt auf GitHub entnommen.
https://github.com/markusschmitt/ml_rapid_tutorial/tree/master/unsupervised_learning

In [ ]:
using CSV, Statistics, PyPlot, DataFrames, Random
# Seed setzen
Random.seed!(1)

## Import der Daten

In [ ]:
# DataFrame importieren
v_data_2=DataFrame(CSV.File("C:\\Users\\cpryg\\Documents\\Bachelorarbeit\\data_2.csv"))
# DataFrame in Matrix konvertieren
v_data_2=convert(Matrix,v_data_2);
# Shuffle der Daten
s=shuffle(collect(1:size(v_data_2)[2]))
v_data_shuffled_2=zeros(size(v_data_2))
for j in 1:size(v_data_2)[2]
    v_data_shuffled_2[:,j]=v_data_2[:,s[j]]
end
v_training_2=v_data_shuffled_2

In [ ]:
# DataFrame importieren
v_data_3=DataFrame(CSV.File("C:\\Users\\cpryg\\Documents\\Bachelorarbeit\\data_3.csv"))
# DataFrame in Matrix konvertieren
v_data_3=convert(Matrix,v_data_3);
# Shuffle der Daten
s=shuffle(collect(1:size(v_data_3)[2]))
v_data_shuffled_3=zeros(size(v_data_3))
for j in 1:size(v_data_3)[2]
    v_data_shuffled_3[:,j]=v_data_3[:,s[j]]
end
v_training_3=v_data_shuffled_3

In [ ]:
# DataFrame importieren
v_data_5=DataFrame(CSV.File("C:\\Users\\cpryg\\Documents\\Bachelorarbeit\\data_5.csv"))
# DataFrame in Matrix konvertieren
v_data_5=convert(Matrix,v_data_5);
# Shuffle der Daten
s=shuffle(collect(1:size(v_data_5)[2]))
v_data_shuffled_5=zeros(size(v_data_5))
for j in 1:size(v_data_5)[2]
    v_data_shuffled_5[:,j]=v_data_5[:,s[j]]
end
v_training_5=v_data_shuffled_5

## Definition der relevanten Funktionen

In [ ]:
# Sigmoidfunktion definieren
function sigmoid(x)
    s=1 ./(1 .+exp.(-x))
    return s
end

Es wurde auf die Bias-Parameter verzichtet, sodass nur noch die Wechselwirkungsmatrix W erlernt werden muss.

In [ ]:
#p(v_i=1|h) ∀ i direkt berechnen 
function p_v__h(h,W)
    p=sigmoid(W*h)
    return p
end
#p(h_j=1|v) ∀ j direkt berechnen 
function p_h__v(v,W)
    p=sigmoid(transpose(W)*v)
    return p
end

In [ ]:
# Ein Schritt vom Gibbs-Sampling für eine gegebene Konfiguration
function gibbs_step(v,W)
    p_h=p_h__v(v,W)
    h_new=ones(size(W)[2])
    for j in 1:size(W)[2]
        if rand()>p_h[j]
            h_new[j]=-1
        end
    end
    p_v=p_v__h(h_new,W)
    v_new=ones(size(W)[1])
    for i in 1:size(W)[1]
        if rand()>p_v[i]
            v_new[i]=-1
        end
    end
    return v_new
end

In [ ]:
# n Schritte von Gibbs-Sampling für N Konfigurationen 
function Gibbs_step(v,W,n)
    Konfiguration=copy(v)
    for i in 1:size(v)[2]
        for t in 1:n
            Konfiguration[:,i]=gibbs_step(Konfiguration[:,i],W)
        end
    end
    return Konfiguration
end

$$
\langle h_j \rangle_{S}=\frac{1}{|S|}\cdot \sum_{\vec{v}\in S}[p(h_j=1|\vec{v})-p(h_j=-1|\vec{v})]=\frac{1}{|S|}\cdot \sum_{\vec{v}\in S} 2\cdot p(h_j=1|\vec{v})-1
$$
$$
\langle v_i \rangle_{S}=\frac{1}{|S|}\cdot \sum_{\vec{v}\in S}v_i
$$
$$
\langle h_j\cdot v_i \rangle_{S}=\frac{1}{|S|}\cdot \sum_{\vec{v}\in S} v_i \cdot[2\cdot p(h_j=1|\vec{v})-1]
$$
$S$ ist eine beliebige Konfigurationsmenge.

In [ ]:
# Gradientenbestimmung durch Constrative Divergence (CD) 
function CD(v,W,n,η)
    # neue Konfigurationsmenge wird generiert
    v_new=Gibbs_step(v,W,n)
    # Bedingte Wahrscheinlichkeiten für jede Konfiguration vorab bestimmen
    p_h=[p_h__v(v[:,n],W) for n in 1:size(v)[2]]
    p_h_new=[p_h__v(v_new[:,n],W) for n in 1:size(v)[2]]
    for i in 1:size(W)[1]
        for j in 1:size(W)[2]
            for n in 1:size(v)[2]
                # Parameterupdate
                W[i,j]+=η/size(v)[2]*(v[i,n]*(2*p_h[n][j]-1)-v_new[i,n]*(2*p_h_new[n][j]-1))
            end
        end
    end
    return W
end

In [ ]:
# Wechselwirkung mit allen anderen Spins werden in einer Matrix gespeichert
function Wechselwirkung(L;J=1)
    V=zeros(L,L)
    for i in 1:L
        for j in 1:L
            if i!=j
                V[i,j]=J/abs(i-j)^2
            end
        end
    end
    return V
end

# Es wird die mittlere reale Energie einer Menge an Konfigurationen betrachtet 
# V=Wechselwirkungsmatrix
function E(v,V;J=1)
    E=0
    for n in 1:size(v)[2]
        for i in 1:size(v)[1]
            for j in 1:size(v)[1]
                E+=-J/2*v[i,n]*v[j,n]*V[i,j]
            end
        end
    end
    E=E/size(v)[2]
    return E
end


In [ ]:
#=Zur Kontrolle des Trainingsprozesses wird die reale mittlere Energie einiger generierter Konfiguration betrachtet.
Es müssen immer dieselben Konfigurationen aus v_data verwendet werden, um die Entwicklung der Energie zu betrachten.
Es wird das Gibbs-Sampling durchgeführt, um die neuen Konfigurationen zu erhalten.
v ist eine Menge an Konfigurationen, minibatches ist die Anzahl der Minibatches, n_h ist die Größe der verborgenen Schicht=#
function training(v,n_h,n,η,epoch,minibatches)
    # Seed setzen
    Random.seed!(1)
    W=randn(size(v)[1],n_h)
    # Wechselwirkungsmatrix 
    V=Wechselwirkung(size(v)[1])
    # mittlere Energien
    E_training=zeros(epoch+1)
    v_new=Gibbs_step(v[:,1:100],W,n)
    E_training[1]=E(v_new,V)
    # mittlere Energien bei der Initialisierung 
    E_0=E(v,V)
    # Array für die Minibatches erstellen
    r=floor(Int,size(v)[2]/minibatches)
    v_minibatch=zeros(size(v)[1],r)
    for t in 1:epoch
        # Indizes shufflen
        idx=shuffle(collect(1:size(v)[2]))
        i=0
        for n in 1:minibatches
            # Mini-batches bilden
            v_minibatch=v[:,idx[i+1:i+r]]
            #Parameterupdate in den Mini-batches
            W=CD(v,W,n,η)
            i+=r
        end
        # neue Daten nach dem Parameterupdate erstellen 
        v_new=Gibbs_step(v[:,1:100],W,n)
        E_training[t+1]=E(v_new,V)
    end
    E_training.-=E_0
    E_training./=E_0
    return abs.(E_training),W
end

## Training des Modells bei den Temperaturen T=2,3,5

In [ ]:
# Training bei T=2
ΔE_2,W_2_final=training(v_training_2[:,1:5000],500,1,0.5,15,7)

In [ ]:
# Plot des Trainingsverlauf bei T=2
plot(ΔE_2,label="normalverteilt")
legend()
ax = gca()
ax[:set_xticks]([0,2,4,6,8,10,12,14,16])
ax[:tick_params](labelsize=15)
ax[:set_xlabel]("Epochen",fontsize=15)
ax[:set_ylabel]("ΔE",fontsize=15)
ax[:legend](fontsize=15)

In [ ]:
# Training bei T=3
ΔE_3,W_3_final=training(v_training_3[:,1:5000],500,1,0.5,15,7)

In [ ]:
# Plot des Trainingsverlauf bei T=3
plot(ΔE_3,label="normalverteilt")
xlabel("Epochen")
ylabel("ΔE")
legend()
ax = gca()
ax[:set_xticks]([0,2,4,6,8,10,12,14,16])
ax[:tick_params](labelsize=15)
ax[:set_xlabel]("Epochen",fontsize=15)
ax[:set_ylabel]("ΔE",fontsize=15)
ax[:legend](fontsize=15)

In [ ]:
# Training bei T=5
ΔE_5,W_5_final=training(v_training_5[:,1:5000],500,1,0.5,15,7)

In [ ]:
# Plot des Trainingsverlauf bei T=5
plot(ΔE_5,label="normalverteilt")
xlabel("Epochen")
ylabel("ΔE")
legend()
ax = gca()
ax[:set_xticks]([0,2,4,6,8,10,12,14,16])
ax[:tick_params](labelsize=15)
ax[:set_xlabel]("Epochen",fontsize=15)
ax[:set_ylabel]("ΔE",fontsize=15)
ax[:legend](fontsize=15)

## Export der Wechselwirkungsmatrizen

In [ ]:
W_2=DataFrame(W_2_final)
CSV.write("...W_2.csv",W_2)
W_3=DataFrame(W_3_final)
CSV.write("...W_3.csv",W_3)
W_5=DataFrame(W_5_final)
CSV.write("...W_5.csv",W_5)

## Analyse der Korrelation 

Spin-Spin Korrelationsfunktion wird berechnet, indem für jede Konfiguration ausgehend von der mittleren Position y=$\lfloor L/2 \rfloor$ der Spinkette L das Produkt zweier Spins mit dem Abstand |y-x| gebildet wird. Dabei wird der Abstand zu Spins, die rechts und links davon liegen, separat betrachtet. Dieser Wert wird anschließend über alle Konfigurationen gemittelt:
$$
\chi(x)^{(n)}=v_y^{(n)} v_{x}^{(n)} \Rightarrow \chi(x)=\frac{1}{N}\sum_{n=1}^N \chi(x)^{(n)}
$$
Da ab einer gewissen Entfernung von der mittleren Position das Untergrundrauschen dominant wird, werden für die mittlere relative Abweichung der Korrelationen nur Werte in einem kleineren Intervall um die mittlere Position herangezogen.

In [ ]:
# mittlere relative Abweichung der beiden Korrelationsfunktionen
function Δχ(χ_1,χ_2,l,r)
    ΔΧ=abs.((χ_1[250+l:250+r].-χ_2[250+l:250+r])./χ_1[250+l:250+r])
    Δ=mean(ΔΧ)
    return Δ
end

### T=2

In [ ]:
# Plot der Korrelationen
# Seed setzen
Random.seed!(1)
v_2=v_training_2[:,1:5000]
v_new_2=Gibbs_step(v_2,W_2_final,1)
# Korrelation der ursprünglichen Daten
χ_old_2=zeros(size(v_2)[1])
# Korrelation der nach dem Training generierten Daten
χ_new_2=zeros(size(v_2)[1])
y=floor(Int,size(v_2)[1]/2)
for x in 1:size(v_2)[1]
    for n in 1:size(v_2)[2]
        χ_old_2[x]+=1/size(v_2)[2]*v_2[y,n]*v_2[x,n]
        χ_new_2[x]+=1/size(v_2)[2]*v_new_2[y,n]*v_new_2[x,n]
    end
end
x=collect(1:500)
plot(x.-250,χ_new_2,label="χ_gelernt")
plot(x.-250,χ_old_2,label="χ_data")
legend()
ax = gca()
ax[:set_xticks]([-200,-100,0,100,200])
ax[:tick_params](labelsize=15)
ax[:set_xlabel]("Δx",fontsize=15)
ax[:set_ylabel]("χ(Δx)",fontsize=15)
ax[:legend](fontsize=15)

# mittlerer Fehler Δχ=|χ_new-χ_old|/|χ_old|
Δχ_2=Δχ(χ_old_2,χ_new_2,-165,165)
println("Abweichung: $Δχ_2")

In [ ]:
# Plot der Korrelationsabweichung gegen die Intervallgrenzen, um diese festzulegen
Δχ_2_funktion=[Δχ(χ_old_2,χ_new_2,-i,i) for i in 100:170]
plot(collect(100:170),Δχ_2_funktion)
ax = gca()
ax[:set_xticks]([100,125,150,170])
ax[:tick_params](labelsize=20)
ax[:set_xlabel]("|x|",fontsize=20)
ax[:set_ylabel]("Δχ(|x|)",fontsize=20)

In [ ]:
# Logarithmische Auftragung, um exponentielle Abhängigkeit zu erkennen.
x=collect(1:500)
semilogy(x.-250,χ_new_2,label="χ_gelernt")
semilogy(x.-250,χ_old_2,label="χ_data")
legend()
ax = gca()
ax[:set_xticks]([-200,-100,0,100,200])
ax[:tick_params](labelsize=15)
ax[:set_xlabel]("Δx",fontsize=15)
ax[:set_ylabel]("χ(Δx)",fontsize=15)
ax[:legend](fontsize=15)
xlim(-200,200)
ylim(10^-3,1.2)

### T=3

In [ ]:
# Plot der Korrelationen
# Seed setzen
Random.seed!(1)
v_3=v_training_3[:,1:5000]
v_new_3=Gibbs_step(v_3,W_3_final,1)
# Korrelation der ursprünglichen Daten
χ_old_3=zeros(size(v_3)[1])
# Korrelation der nach dem Training generierten Daten
χ_new_3=zeros(size(v_3)[1])
y=floor(Int,size(v_3)[1]/2)
for x in 1:size(v_3)[1]
    for n in 1:size(v_3)[2]
        χ_old_3[x]+=1/size(v_3)[2]*v_3[y,n]*v_3[x,n]
        χ_new_3[x]+=1/size(v_3)[2]*v_new_3[y,n]*v_new_3[x,n]
    end
end
x=collect(1:500)
plot(x.-250,χ_new_3,label="χ_gelernt")
plot(x.-250,χ_old_3,label="χ_data")
legend()
ax = gca()
ax[:set_xticks]([-200,-100,0,100,200])
ax[:tick_params](labelsize=15)
ax[:set_xlabel]("Δx",fontsize=15)
ax[:set_ylabel]("χ(Δx)",fontsize=15)
ax[:legend](fontsize=15)
# mittlerer Fehler Δχ=|χ_new-χ_old|/|χ_old|
Δχ_3=Δχ(χ_old_3,χ_new_3,-16,16)
println("Abweichung: $Δχ_3")

In [ ]:
# Plot der Korrelationsabweichung gegen die Intervallgrenzen, um diese festzulegen
Δχ_3_funktion=[Δχ(χ_old_3,χ_new_3,-i,i) for i in 5:30]
plot(collect(5:30),Δχ_3_funktion)
ax = gca()
ax[:set_xticks]([5,10,15,20,25,30])
ax[:tick_params](labelsize=20)
ax[:set_xlabel]("|x|",fontsize=20)
ax[:set_ylabel]("Δχ(|x|)",fontsize=20)

In [ ]:
# Logarithmische Auftragung, um exponentielle Abhängigkeit zu erkennen.
x=collect(1:500)
semilogy(x.-250,χ_new_3,label="χ_gelernt")
semilogy(x.-250,χ_old_3,label="χ_data")
legend()
ax = gca()
ax[:set_xticks]([-40,-40,0,20,40])
ax[:tick_params](labelsize=15)
ax[:set_xlabel]("Δx",fontsize=15)
ax[:set_ylabel]("χ(Δx)",fontsize=15)
ax[:legend](fontsize=15)
xlim(-50,50)
ylim(10^-3,1.2)

### T=5

In [ ]:
# Plot der Korrelationen
# Seed setzen
Random.seed!(1)
v_5=v_training_5[:,1:5000]
v_new_5=Gibbs_step(v_5,W_5_final,1)
# Korrelation der ursprünglichen Daten
χ_old_5=zeros(size(v_5)[1])
# Korrelation der nach dem Training generierten Daten
χ_new_5=zeros(size(v_5)[1])
y=floor(Int,size(v_5)[1]/2)
for x in 1:size(v_5)[1]
    for n in 1:size(v_5)[2]
        χ_old_5[x]+=1/size(v_5)[2]*v_5[y,n]*v_5[x,n]
        χ_new_5[x]+=1/size(v_5)[2]*v_new_5[y,n]*v_new_5[x,n]
    end
end
x=collect(1:500)
plot(x.-250,χ_new_5,label="χ_gelernt")
plot(x.-250,χ_old_5,label="χ_data")
legend()
ax = gca()
ax[:set_xticks]([-200,-100,0,100,200])
ax[:tick_params](labelsize=15)
ax[:set_xlabel]("Δx",fontsize=15)
ax[:set_ylabel]("χ(Δx)",fontsize=15)
ax[:legend](fontsize=15)

# mittlerer Fehler Δχ=|χ_new-χ_old|/|χ_old|
Δχ_5=Δχ(χ_old_5,χ_new_5,-5,5)
println("Abweichung: $Δχ_5")

In [ ]:
# Plot der Korrelationsabweichung gegen die Intervallgrenzen, um diese festzulegen
Δχ_5_funktion=[Δχ(χ_old_5,χ_new_5,-i,i) for i in 1:10]
plot(collect(1:10),Δχ_5_funktion)
ax = gca()
ax[:set_xticks]([1,3,5,7,9])
ax[:tick_params](labelsize=20)
ax[:set_xlabel]("|x|",fontsize=20)
ax[:set_ylabel]("Δχ(|x|)",fontsize=20)

In [ ]:
# Logarithmische Auftragung, um exponentielle Abhängigkeit zu erkennen.
x=collect(1:500)
semilogy(x.-250,χ_new_5,label="χ_gelernt")
semilogy(x.-250,χ_old_5,label="χ_data")
legend()
ax = gca()
ax[:set_xticks]([-15,-10,-5,0,5,10,15])
ax[:tick_params](labelsize=15)
ax[:set_xlabel]("Δx",fontsize=15)
ax[:set_ylabel]("χ(Δx)",fontsize=15)
ax[:legend](fontsize=15)
xlim(-15,15)
ylim(10^-3,1.2)

## Korrelationsmatrix

In [ ]:
# Korrelationsmatrix erstellen
function Korrelationsmatrix(v)
    χ=zeros(size(v)[1],size(v)[1])
    for i in 1:size(v)[1]
        for j in 1:size(v)[1]
            χ[i,j]=mean([v[i,n]*v[j,n] for n in 1:size(v)[2]])
        end
    end
    # logarithmische Darstellung
    imshow(log.(abs.(χ)))
    return nothing
end

In [ ]:
# T=2
Korrelationsmatrix(v_training_2[:,1:5000])
ax = gca()
ax[:set_xticks]([1,500])
ax[:set_yticks]([1,500])
ax[:tick_params](labelsize=20);

In [ ]:
# T=3
Korrelationsmatrix(v_training_3[:,1:5000])
ax = gca()
ax[:set_xticks]([1,500])
ax[:set_yticks]([1,500])
ax[:tick_params](labelsize=20);

In [ ]:
# T=5
Korrelationsmatrix(v_training_5[:,1:5000])
ax = gca()
ax[:set_xticks]([1,500])
ax[:set_yticks]([1,500])
ax[:tick_params](labelsize=20);